<a href="https://colab.research.google.com/github/Whoami666/HSE_Lab/blob/main/schiz_models_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import warnings

In [1]:
import pandas as pd
import numpy as np

In [ ]:
!pip install scikit-optimize

In [15]:
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

from skopt import BayesSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

In [54]:
scoring = ['recall', 'accuracy', 'f1', 'roc_auc']

In [3]:
df = pd.read_excel('/content/df_standard_cleaned_21feb.xlsx')   
df = df.set_index('ID')
df.head(1)

,sex,tds,chroma_cens_avg,chroma_cens_std,chroma_cqt_avg,chroma_cqt_std,chroma_stft_avg,chroma_stft_std,melspectrogram_avg,melspectrogram_std,...,spectral_flatness_std,spectral_rolloff_avg,spectral_rolloff_std,tempogram_avg,tempogram_std,tonnetz_avg,tonnetz_std,zero_crossing_rate_avg,zero_crossing_rate_std,tds_binary
ID,,,,,,,,,,,,,,,,,,,,,
PD-003,1,0,0.685706,0.428088,0.425039,0.296394,0.339246,0.434412,0.327867,0.348849,...,0.31499,0.397736,0.254215,0.747885,0.719312,0.691479,0.329593,0.197304,0.281692,0


Non-binary random forest

In [7]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [51]:
warnings.filterwarnings('ignore')


opt = BayesSearchCV(
    RandomForestClassifier(),
    {
        'n_estimators':  (100, 1000),
        'max_depth': (10, 100),
        'min_samples_split': np.linspace(0.0001, 1, 10),
        'min_samples_leaf': np.linspace(0.0001, 0.5, 10),
    },
    n_iter=32,
    cv=3
)

opt.fit(X_train, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_test, y_test))
print("best_params: %s" % opt.best_params_)

warnings.filterwarnings('default')

val. score: 0.8912655971479501
test score: 0.8823529411764706
best_params: OrderedDict([('max_depth', 25), ('min_samples_leaf', 0.05564444444444445), ('min_samples_split', 0.33340000000000003), ('n_estimators', 132)])


In [52]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = opt.best_params_['n_estimators'], 
                          #   class_weight = "balanced",
                             max_depth = opt.best_params_['max_depth'],
                             min_samples_split = opt.best_params_['min_samples_split'],
                             min_samples_leaf = opt.best_params_['min_samples_leaf'])
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94        60
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         2

    accuracy                           0.88        68
   macro avg       0.29      0.33      0.31        68
weighted avg       0.78      0.88      0.83        68



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Binary

In [57]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds_binary # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [58]:
warnings.filterwarnings('ignore')

opt = BayesSearchCV(
    RandomForestClassifier(),
    {
        'n_estimators':  (100, 1000),
        'max_depth': (10, 100),
        'min_samples_split': np.linspace(0.0001, 1, 10), #add integers greater than 1
        'min_samples_leaf': np.linspace(0.0001, 0.5, 10), #add integers greater than 1
    },
    n_iter=32,
    cv=3
)

opt.fit(X_train, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_test, y_test))
print("best_params: %s" % opt.best_params_)

val. score: 0.9013666072489602
test score: 0.9117647058823529
best_params: OrderedDict([('max_depth', 100), ('min_samples_leaf', 0.0001), ('min_samples_split', 0.0001), ('n_estimators', 137)])


In [59]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = opt.best_params_['n_estimators'], 
                          #   class_weight = "balanced",
                             max_depth = opt.best_params_['max_depth'],
                             min_samples_split = opt.best_params_['min_samples_split'],
                             min_samples_leaf = opt.best_params_['min_samples_leaf'])
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95        60
           1       0.75      0.38      0.50         8

    accuracy                           0.91        68
   macro avg       0.84      0.68      0.73        68
weighted avg       0.90      0.91      0.90        68



In [60]:
print(classification_report(y_test,y_pred))
val_acc = accuracy_score(y_pred, y_test)
print(val_acc)

              precision    recall  f1-score   support

           0       0.92      0.98      0.95        60
           1       0.75      0.38      0.50         8

    accuracy                           0.91        68
   macro avg       0.84      0.68      0.73        68
weighted avg       0.90      0.91      0.90        68

0.9117647058823529


In [61]:
cross_validate(rfc, X, y, scoring=scoring)

{'fit_time': array([0.19654107, 0.18987107, 0.18359184, 0.18755507, 0.18096519]),
 'score_time': array([0.0322926 , 0.03190112, 0.03135872, 0.0306282 , 0.02975464]),
 'test_recall': array([0.25      , 0.        , 0.5       , 0.25      , 0.66666667]),
 'test_accuracy': array([0.88235294, 0.82352941, 0.94117647, 0.88235294, 0.87878788]),
 'test_f1': array([0.33333333, 0.        , 0.66666667, 0.33333333, 0.5       ]),
 'test_roc_auc': array([0.90833333, 0.80833333, 0.82083333, 0.8875    , 0.74444444])}

In [ ]:
#Notes
integers = np.array([i for i in range(1, 50)])
min_samples_split_steps = list(np.linspace(0.0, 1, 50)).append(integers)

#min_samples_leaf_steps = np.append(np.linspace(0.0, 0.5, 50), integers)